Step 1: Setup Colab
Install the required libraries.

In [ ]:
!pip install fastapi[all] gradio scikit-learn uvicorn nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.5/950.5 kB 21.3 MB/s eta 0:00:00


 Step 2: Train the Iris model
We’ll use scikit-learn to train a simple classifier and save it.

if petal_length < 2:

    species = 'Setosa'

elif petal_width < 1.8:

    species = 'Versicolor'

else:

    species = 'Virginica'

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load Iris data
iris = load_iris()
X, y = iris.data, iris.target

# Train the model
clf = RandomForestClassifier()
clf.fit(X, y)

# Save the model
with open("iris_model.pkl", "wb") as f:
    pickle.dump(clf, f)


 Step 3: Create the FastAPI app
We’ll create a FastAPI app with a Gradio interface to serve predictions.

In [ ]:
%%writefile app.py
from fastapi import FastAPI
import gradio as gr
import pickle
import numpy as np

# Load the model
with open("iris_model.pkl", "rb") as f:
    model = pickle.load(f)

# Define prediction function
def predict(sepal_length, sepal_width, petal_length, petal_width):
    x = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
    y_pred = model.predict(x)[0]
    return f"Predicted class: {y_pred}"

# Create Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Number(label="Sepal Length"),
        gr.Number(label="Sepal Width"),
        gr.Number(label="Petal Length"),
        gr.Number(label="Petal Width")
    ],
    outputs="text",
    title="Iris Classifier"
)

# Create FastAPI app
app = FastAPI()

# Mount Gradio on FastAPI
app = gr.mount_gradio_app(app, interface, path="/")


Writing app.py


Step 4: Run the app in Colab

Use ngrok to tunnel the app and get a public URL.

Install & set up ngrok:

In [ ]:
uvicorn.run("app:app", host="0.0.0.0", port=8000)


INFO:     Started server process [255]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     103.224.35.45:0 - "GET / HTTP/1.1" 307 Temporary Redirect
INFO:     103.224.35.45:0 - "GET // HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/index-Dz5CWoA7.css HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/index-Ccc2t4AG.js HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/svelte/svelte.js HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/Index-BzI0zy4H.js HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/Embed-yYAfXBkj.js HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/index-BFBcOI-E.js HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/DownloadLink.svelte_svelte_type_style_lang-C_5UIfol.js HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/IconButton-C_HS7fTi.js HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/Index-BJ_RfjVB.css HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/MarkdownCode-Dw1-H2nG.css HTTP/1.1" 200 OK
INFO:     103.224.35.45:0 - "GET //assets/Clear-By3xiIwg.j

In [ ]:
!pip install pyngrok
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Allow nested event loops
nest_asyncio.apply()

# 🔑 Set your ngrok authtoken here
ngrok.set_auth_token("2zpTw0AasvxhVZGU5klund7atRH_4mHs7dfgfNgmKjfG1gSSq")

# Expose port 8000
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")


Public URL: NgrokTunnel: "https://57e692920548.ngrok-free.app" -> "http://localhost:8000"
